<a href="https://colab.research.google.com/github/minurasam/Brain-Tumor-Detection-CNN/blob/main/CNN_impl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cpu


In [38]:
num_epochs = 4
batch_Size = 5
learning_rate = 0.01

# Load the Dataset

In [39]:
# Dataset has PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


In [40]:
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_Size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_Size, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


#Implement the Convolutional Neural Network

In [41]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    #input channels = 3 as you 3 channels in the input image
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    # self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 16 * 5 * 5)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

my_nn = ConvNet()
print(my_nn)

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [42]:
# # Equates to one random 28x28 image
# random_data = torch.rand((1, 1, 28, 28))

# my_nn = ConvNet()
# result = my_nn(random_data)
# print (result)

model = ConvNet().to(device)

In [43]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i+1) % 2500 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/4], Step [2500/10000], loss: 2.3123
Epoch [1/4], Step [5000/10000], loss: 2.3025
Epoch [1/4], Step [7500/10000], loss: 2.2910
Epoch [1/4], Step [10000/10000], loss: 2.3517
Epoch [2/4], Step [2500/10000], loss: 2.2884
Epoch [2/4], Step [5000/10000], loss: 2.3207
Epoch [2/4], Step [7500/10000], loss: 2.3610
Epoch [2/4], Step [10000/10000], loss: 2.2420
Epoch [3/4], Step [2500/10000], loss: 2.2983
Epoch [3/4], Step [5000/10000], loss: 2.3242
Epoch [3/4], Step [7500/10000], loss: 2.3655
Epoch [3/4], Step [10000/10000], loss: 2.2744


In [27]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels - labels.to(device)
    outputs = model(images)
    # max returns (value, index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_Size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network is: {acc} %')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] /n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network is: 59.51 %
Accuracy of plane: 74.0 %
Accuracy of car: 68.9 %
Accuracy of bird: 44.7 %
Accuracy of cat: 29.7 %
Accuracy of deer: 58.1 %
Accuracy of dog: 46.2 %
Accuracy of frog: 67.7 %
Accuracy of horse: 68.7 %
Accuracy of ship: 71.2 %
Accuracy of truck: 65.9 %
